In [2]:
__author__ = "Farzaneh Soltanzadeh"
__email__ = "f.soltanzadeh.f@gmail.com"
__organization__ = "Coursera"
__date__ = "2024-12"

Import Libraries 📚

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import plotly.graph_objs as go
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

Dataset 📦 

In [14]:
df_train = pd.read_csv('Data/laptop_price - dataset.csv')
print(len(df_train))
df_train.head()

1275


,Company,Product,TypeName,Inches,ScreenResolution,CPU_Company,CPU_Type,CPU_Frequency (GHz),RAM (GB),Memory,GPU_Company,GPU_Type,OpSys,Weight (kg),Price (Euro)
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel,Core i5,2.3,8,128GB SSD,Intel,Iris Plus Graphics 640,macOS,1.37,1339.69
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel,Core i5,1.8,8,128GB Flash Storage,Intel,HD Graphics 6000,macOS,1.34,898.94
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel,Core i5 7200U,2.5,8,256GB SSD,Intel,HD Graphics 620,No OS,1.86,575.00
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel,Core i7,2.7,16,512GB SSD,AMD,Radeon Pro 455,macOS,1.83,2537.45
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel,Core i5,3.1,8,256GB SSD,Intel,Iris Plus Graphics 650,macOS,1.37,1803.60


In [11]:
df_train = shuffle(df_train, random_state=1)

# (60% train, 20% val, 20% test)
train_set, temp_set = train_test_split(df_train, test_size=0.4, random_state=1)
val_set, test_set = train_test_split(temp_set, test_size=0.5, random_state=1)